In [1]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [ ]:
# Create authentication object and log in to desired user account (if not already authenticated)
auth = TokenAuthentication(
    token = "XXXX",
    server = "XXXX",
    skip_tls = False
)
auth.login()

In [2]:
# Create our cluster and submit appwrapper
namespace = "default"
cluster_name = "hfgputest-1"
local_interactive = True

cluster = Cluster(ClusterConfiguration(local_interactive=local_interactive, namespace=namespace, name=cluster_name, min_worker=1, max_worker=1, min_cpus=1, max_cpus=1, min_memory=4, max_memory=4, gpu=0, instascale=False, machine_types=["m5.xlarge", "p3.8xlarge"]))

Written to: hfgputest-1.yaml


In [3]:
cluster.up()

In [4]:
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster up and running!


### Connect via the rayclient route

In [11]:
import openshift as oc
from codeflare_sdk.utils import generate_cert

if local_interactive:
    generate_cert.generate_tls_cert(cluster_name, namespace)
    generate_cert.export_env(cluster_name, namespace)

with oc.project(namespace):
    routes=oc.selector("route").objects()
    rayclient_url=""
    for r in routes:
        if "rayclient" in r.name():
            rayclient_url=r.model.spec.host
print(rayclient_url)

rayclient-hfgputest-1-default.apps.tedbig412.cp.fyre.ibm.com


In [12]:
import ray

ray.shutdown()
ray.init(address=f"ray://{rayclient_url}", logging_level="DEBUG")

2023-05-31 14:12:37,816	INFO client_builder.py:251 -- Passing the following kwargs to ray.init() on the server: logging_level
2023-05-31 14:12:37,820	DEBUG worker.py:378 -- client gRPC channel state change: ChannelConnectivity.IDLE
2023-05-31 14:12:38,034	DEBUG worker.py:378 -- client gRPC channel state change: ChannelConnectivity.CONNECTING
2023-05-31 14:12:38,246	DEBUG worker.py:378 -- client gRPC channel state change: ChannelConnectivity.READY
2023-05-31 14:12:38,290	DEBUG worker.py:807 -- Pinging server.
2023-05-31 14:12:40,521	DEBUG worker.py:640 -- Retaining 00ffffffffffffffffffffffffffffffffffffff0100000001000000
2023-05-31 14:12:40,523	DEBUG worker.py:564 -- Scheduling task get_dashboard_url 0 b'\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x01\x00\x00\x00\x01\x00\x00\x00'
2023-05-31 14:12:40,535	DEBUG worker.py:640 -- Retaining c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000
2023-05-31 14:12:41,379	DEBUG worker.py:636 -- Releasing c8

Python version:,3.8.13
Ray version:,2.1.0
Dashboard:,http://10.254.12.141:8265


In [13]:
import math
import ray

@ray.remote
def heavy_calculation_part(num_iterations):
    result = 0.0
    for i in range(num_iterations):
        for j in range(num_iterations):
            for k in range(num_iterations):
                result += math.sin(i) * math.cos(j) * math.tan(k)
    return result
@ray.remote
def heavy_calculation(num_iterations):
    results = ray.get([heavy_calculation_part.remote(num_iterations//30) for _ in range(30)])
    return sum(results)


In [14]:
ref = heavy_calculation.remote(3000)

2023-05-31 14:13:29,868	DEBUG worker.py:640 -- Retaining 00ffffffffffffffffffffffffffffffffffffff0100000002000000
2023-05-31 14:13:29,870	DEBUG worker.py:564 -- Scheduling task heavy_calculation 0 b'\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x01\x00\x00\x00\x02\x00\x00\x00'


In [15]:
ray.get(ref)

2023-05-31 14:13:32,643	DEBUG worker.py:640 -- Retaining 16310a0f0a45af5cffffffffffffffffffffffff0100000001000000
2023-05-31 14:13:34,677	DEBUG worker.py:439 -- Internal retry for get [ClientObjectRef(16310a0f0a45af5cffffffffffffffffffffffff0100000001000000)]


1789.4644387076714

In [16]:
ray.cancel(ref)
ray.shutdown()

2023-05-31 14:13:37,659	DEBUG dataclient.py:287 -- Got unawaited response connection_cleanup {
}

2023-05-31 14:13:38,681	DEBUG dataclient.py:278 -- Shutting down data channel.


In [17]:
cluster.down()